In [11]:
model_path = '테스트모델(K오류).h5'
model_path_inference = '테스트모델(추론용).h5'

annotation_path = './yolo_train.txt'
classes_path = './yolo_classes.txt'
anchors_path = './yolo_anchors.txt'

import os # Protocol Buffers to use Python rather than C++
os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'

## GPU에 할당(사용)되는 메모리 크기 제한
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
gpu_memory_limit=1024*12 # only allocate 12GB of memory on the gpus[0], i.e. first GPU
if gpus:
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],[tf.config.experimental.VirtualDeviceConfiguration(memory_limit=gpu_memory_limit)])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print("GPU memory allocation(",gpu_memory_limit,")",
          "# of Physical GPU(",len(gpus),") # of Logical GPU(",len(logical_gpus),")")
  except RuntimeError as e:
    print(e) # Virtual devices must be set before GPUs have been initialized
else:
   print("There is no GPU to use.")

import sys
sys.path.append("./")

import tensorflow as tf
import numpy as np
from keras.layers import Input, Lambda
from keras.models import Model

from yolo3_model_for_training import yolo_body, yolo_loss

# Print the current date and time in the format:
# "YYYY-MM-DD HH:MM:SS.microseconds"
import datetime
def print_current_datetime(text=""):
    datetime_string = datetime.datetime.now().strftime("%Y-%m-%dT%H:%M:%S.%f")
    print("{} @ CDT({})".format(text,datetime_string))

print_current_datetime()

def yolo3_model_to_convert(input_shape, anchors, num_classes):

    image_input = Input(shape=(None, None, 3))
    num_anchors = len(anchors)

    model_body = yolo_body(image_input, num_anchors//3, num_classes)

    h, w = input_shape
    y_true = [Input(shape=(h//{0:32, 1:16, 2:8}[l], w//{0:32, 1:16, 2:8}[l], num_anchors//3, num_classes+5)) for l in range(3)]

    model_loss = Lambda(yolo_loss, output_shape=(1,), name='yolo_loss',
        arguments={'anchors': anchors, 'num_classes': num_classes, 'ignore_thresh': 0.5})([*model_body.output, *y_true])
    model = Model([model_body.input, *y_true], model_loss)

    return model

def get_classes(classes_path):
    with open(classes_path) as f:
        class_names = f.readlines()
    class_names = [c.strip() for c in class_names]
    return class_names

def get_anchors(anchors_path):
    with open(anchors_path) as f:
        anchors = f.readline()
    anchors = [float(x) for x in anchors.split(',')]
    return np.array(anchors).reshape(-1, 2)

class_names = get_classes(classes_path)
num_classes = len(class_names)
anchors = get_anchors(anchors_path)

input_shape = (416,416) # multiple of 32, hw

# To prevent yolo_loss() NameError: name 'K' is not defined
inference_model = yolo_body(Input(shape=(None, None, 3)), len(anchors)//3, num_classes)
inference_model.load_weights(model_path)
inference_model.save(model_path_inference)

print_current_datetime()

There is no GPU to use.
 @ CDT(2023-06-21T14:03:09.653859)
 @ CDT(2023-06-21T14:03:20.753616)
